In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1) insert preprocess and evalcategory functions from prior notebook. 
Note the process_text_class(.) has been updated to include an additional output and evalcategory(.) has been updated to improve address of NaN

In [0]:
#process_numerical_class(mdf_train, mdf_test, column)
#function to normalize data to mean of 0 and standard deviation of 1 from training distribution
#takes as arguement pandas dataframe of training and test data (mdf_train), (mdf_test)\
#and the name of the column string ('column') 
#replaces missing or improperly formatted data with mean of remaining values
#replaces original specified column in dataframe
#returns transformed dataframe

#expect this approach works better when the numerical distribution is thin tailed
#if only have training but not test data handy, use same training data for both dataframe inputs

#imports
from pandas import Series
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def process_numerical_class(mdf_train, mdf_test, column):
     
    
  #convert all values to either numeric or NaN
  mdf_train[column] = pd.to_numeric(mdf_train[column], errors='coerce')
  mdf_test[column] = pd.to_numeric(mdf_test[column], errors='coerce')

  #get mean of training data
  mean = mdf_train[column].mean()    

  #replace missing data with training set mean
  mdf_train[column] = mdf_train[column].fillna(mean)
  mdf_test[column] = mdf_test[column].fillna(mean)

  #subtract mean from column for both train and test
  mdf_train[column] = mdf_train[column] - mean
  mdf_test[column] = mdf_test[column] - mean

  #get standard deviation of training data
  std = mdf_train[column].std()

  #divide column values by std for both training and test data
  mdf_train[column] = mdf_train[column] / std
  mdf_test[column] = mdf_test[column] / std


  return mdf_train, mdf_test
  

  
#process_binary_class(mdf, column, missing)
#converts binary classification values to 0 or 1
#takes as arguement a pandas dataframe (mdf), \
#the name of the column string ('column') \
#and the string classification to assign to missing data ('missing')
#replaces original specified column in dataframe
#returns transformed dataframe

#missing category must be identical to one of the two existing categories
#returns error message if more than two categories remain


def process_binary_class(mdf, column, missing):
    
  #replace missing data with specified classification
  mdf[column] = mdf[column].fillna(missing)

  #if more than two remaining classifications, return error message    
  if len(mdf[column].unique()) > 2:
      print('ERROR: number of categories in column for process_binary_class() call >2')
      return mdf

  #convert column to binary 0/1 classification
  lb = preprocessing.LabelBinarizer()
  mdf[column] = lb.fit_transform(mdf[column])

  return mdf

  
#process_text_class(mdf_train, mdf_test, column)
#preprocess column with text classifications
#takes as arguement two pandas dataframe containing training and test data respectively 
#(mdf_train, mdf_test), and the name of the column string ('column')

#note this trains both training and test data simultaneously due to unique treatment if any category
#missing from training set but not from test set to ensure consistent formatting 

#deletes the original column from master dataframe and
#replaces with onehot encodings
#with columns named after column_ + text classifications
#missing data replaced with category label 'missing'+column
#any categories missing from the training set removed from test set
#any category present in training but missing from test set given a column of zeros for consistent formatting
#ensures order of all new columns consistent between both sets
#returns two transformed dataframe (mdf_train, mdf_test) \
#and a list of the new column names (textcolumns)

#if only have training but not test data handy, use same training data for both dataframe inputs


def process_text_class(mdf_train, mdf_test, column):

  #replace NA with a dummy variable
  mdf_train[column] = mdf_train[column].fillna('_missing')
  mdf_test[column] = mdf_test[column].fillna('_missing')


  #extract categories for column labels
  #note that .unique() extracts the labels as a numpy array
  labels_train = mdf_train[column].unique()
  labels_train.sort(axis=0)
  labels_test = mdf_test[column].unique()
  labels_test.sort(axis=0)

  #transform text classifications to numerical id
  encoder = LabelEncoder()
  cat_train = mdf_train[column]
  cat_train_encoded = encoder.fit_transform(cat_train)

  cat_test = mdf_test[column]
  cat_test_encoded = encoder.fit_transform(cat_test)


  #apply onehotencoding
  onehotencoder = OneHotEncoder()
  cat_train_1hot = onehotencoder.fit_transform(cat_train_encoded.reshape(-1,1))
  cat_test_1hot = onehotencoder.fit_transform(cat_test_encoded.reshape(-1,1))

  #append column header name to each category listing
  #note the iteration is over a numpy array hence the [...] approach
  labels_train[...] = column + '_' + labels_train[...]
  labels_test[...] = column + '_' + labels_test[...]


  #convert sparse array to pandas dataframe with column labels
  df_train_cat = pd.DataFrame(cat_train_1hot.toarray(), columns=labels_train)
  df_test_cat = pd.DataFrame(cat_test_1hot.toarray(), columns=labels_test)


  #Get missing columns in test set that are present in training set
  missing_cols = set( df_train_cat.columns ) - set( df_test_cat.columns )
  #Add a missing column in test set with default value equal to 0
  for c in missing_cols:
      df_test_cat[c] = 0
  #Ensure the order of column in the test set is in the same order than in train set
  #Note this also removes categories in test set that aren't present in training set
  df_test_cat = df_test_cat[df_train_cat.columns]


  #concatinate the sparse set with the rest of our training data
  mdf_train = pd.concat([df_train_cat, mdf_train], axis=1)
  mdf_test = pd.concat([df_test_cat, mdf_test], axis=1)


  #delete original column from training data
  del mdf_train[column]    
  del mdf_test[column]
  
  #**************************
  #to support the ML infill method, I am updating this function to include an \
  #additional output of a list of the created column names
  
  textcolumns = labels_train
  
  #***************
  

  return mdf_train, mdf_test, textcolumns
  

In [0]:
#process_time_class(mdf_train, mdf_test, column)
#preprocess column with time classifications
#takes as arguement two pandas dataframe containing training and test data respectively 
#(mdf_train, mdf_test), and the name of the column string ('column')

#note this trains both training and test data simultaneously due to unique treatment if any category
#missing from training set but not from test set to ensure consistent formatting 

#deletes the original column from master dataframe and
#replaces with distinct columns for year, month, day, hour, minute, second
#each normalized to the mean and std, with missing values plugged with the mean
#with columns named after column_ + time category
#returns two transformed dataframe (mdf_train, mdf_test)

#if only have training but not test data handy, use same training data for both dataframe inputs

import datetime as dt

def process_time_class(mdf_train, mdf_test, column):
  
  #apply pd.to_datetime to column, note that the errors = 'coerce' needed for messy data
  mdf_train[column] = pd.to_datetime(mdf_train[column], errors = 'coerce')
  mdf_test[column] = pd.to_datetime(mdf_test[column], errors = 'coerce')
  
  #mdf_train[column].replace(-np.Inf, np.nan)
  #mdf_test[column].replace(-np.Inf, np.nan)
  
  #get mean of various categories of datetime objects to use to plug in missing cells
  meanyear = mdf_train[column].dt.year.mean()    
  meanmonth = mdf_train[column].dt.month.mean()
  meanday = mdf_train[column].dt.day.mean()
  meanhour = mdf_train[column].dt.hour.mean()
  meanminute = mdf_train[column].dt.minute.mean()
  meansecond = mdf_train[column].dt.second.mean()
  
  #get standard deviation of training data
  stdyear = mdf_train[column].dt.year.std()  
  stdmonth = mdf_train[column].dt.month.std()
  stdday = mdf_train[column].dt.day.std()
  stdhour = mdf_train[column].dt.hour.std()
  stdminute = mdf_train[column].dt.minute.std()
  stdsecond = mdf_train[column].dt.second.std()
  
  
  #create new columns for each category in train set
  mdf_train[column + '_year'] = mdf_train[column].dt.year
  mdf_train[column + '_month'] = mdf_train[column].dt.month
  mdf_train[column + '_day'] = mdf_train[column].dt.day
  mdf_train[column + '_hour'] = mdf_train[column].dt.hour
  mdf_train[column + '_minute'] = mdf_train[column].dt.minute
  mdf_train[column + '_second'] = mdf_train[column].dt.second
  
  #do same for test set
  mdf_test[column + '_year'] = mdf_test[column].dt.year
  mdf_test[column + '_month'] = mdf_test[column].dt.month
  mdf_test[column + '_day'] = mdf_test[column].dt.day
  mdf_test[column + '_hour'] = mdf_test[column].dt.hour
  mdf_test[column + '_minute'] = mdf_test[column].dt.minute 
  mdf_test[column + '_second'] = mdf_test[column].dt.second
  

  #replace missing data with training set mean
  mdf_train[column + '_year'] = mdf_train[column + '_year'].fillna(meanyear)
  mdf_train[column + '_month'] = mdf_train[column + '_month'].fillna(meanmonth)
  mdf_train[column + '_day'] = mdf_train[column + '_day'].fillna(meanday)
  mdf_train[column + '_hour'] = mdf_train[column + '_hour'].fillna(meanhour)
  mdf_train[column + '_minute'] = mdf_train[column + '_minute'].fillna(meanminute)
  mdf_train[column + '_second'] = mdf_train[column + '_second'].fillna(meansecond)
  
  #do same for test set
  mdf_test[column + '_year'] = mdf_test[column + '_year'].fillna(meanyear)
  mdf_test[column + '_month'] = mdf_test[column + '_month'].fillna(meanmonth)
  mdf_test[column + '_day'] = mdf_test[column + '_day'].fillna(meanday)
  mdf_test[column + '_hour'] = mdf_test[column + '_hour'].fillna(meanhour)
  mdf_test[column + '_minute'] = mdf_test[column + '_minute'].fillna(meanminute)
  mdf_test[column + '_second'] = mdf_test[column + '_second'].fillna(meansecond)
  
  #subtract mean from column for both train and test
  mdf_train[column + '_year'] = mdf_train[column + '_year'] - meanyear
  mdf_train[column + '_month'] = mdf_train[column + '_month'] - meanmonth
  mdf_train[column + '_day'] = mdf_train[column + '_day'] - meanday
  mdf_train[column + '_hour'] = mdf_train[column + '_hour'] - meanhour
  mdf_train[column + '_minute'] = mdf_train[column + '_minute'] - meanminute
  mdf_train[column + '_second'] = mdf_train[column + '_second'] - meansecond
  
  mdf_test[column + '_year'] = mdf_test[column + '_year'] - meanyear
  mdf_test[column + '_month'] = mdf_test[column + '_month'] - meanmonth
  mdf_test[column + '_day'] = mdf_test[column + '_day'] - meanday
  mdf_test[column + '_hour'] = mdf_test[column + '_hour'] - meanhour
  mdf_test[column + '_minute'] = mdf_test[column + '_minute'] - meanminute
  mdf_test[column + '_second'] = mdf_test[column + '_second'] - meansecond
  
  
  #divide column values by std for both training and test data
  mdf_train[column + '_year'] = mdf_train[column + '_year'] / stdyear
  mdf_train[column + '_month'] = mdf_train[column + '_month'] / stdmonth
  mdf_train[column + '_day'] = mdf_train[column + '_day'] / stdday
  mdf_train[column + '_hour'] = mdf_train[column + '_hour'] / stdhour
  mdf_train[column + '_minute'] = mdf_train[column + '_minute'] / stdminute
  mdf_train[column + '_second'] = mdf_train[column + '_second'] / stdsecond
  
  mdf_test[column + '_year'] = mdf_test[column + '_year'] / stdyear
  mdf_test[column + '_month'] = mdf_test[column + '_month'] / stdmonth
  mdf_test[column + '_day'] = mdf_test[column + '_day'] / stdday
  mdf_test[column + '_hour'] = mdf_test[column + '_hour'] / stdhour
  mdf_test[column + '_minute'] = mdf_test[column + '_minute'] / stdminute
  mdf_test[column + '_second'] = mdf_test[column + '_second'] / stdsecond
  
  
  #now replace NaN with 0
  mdf_train[column + '_year'] = mdf_train[column + '_year'].fillna(0)
  mdf_train[column + '_month'] = mdf_train[column + '_month'].fillna(0)
  mdf_train[column + '_day'] = mdf_train[column + '_day'].fillna(0)
  mdf_train[column + '_hour'] = mdf_train[column + '_hour'].fillna(0)
  mdf_train[column + '_minute'] = mdf_train[column + '_minute'].fillna(0)
  mdf_train[column + '_second'] = mdf_train[column + '_second'].fillna(0)
  
  #do same for test set
  mdf_test[column + '_year'] = mdf_test[column + '_year'].fillna(0)
  mdf_test[column + '_month'] = mdf_test[column + '_month'].fillna(0)
  mdf_test[column + '_day'] = mdf_test[column + '_day'].fillna(0)
  mdf_test[column + '_hour'] = mdf_test[column + '_hour'].fillna(0)
  mdf_test[column + '_minute'] = mdf_test[column + '_minute'].fillna(0)
  mdf_test[column + '_second'] = mdf_test[column + '_second'].fillna(0)
  
  
  
  #this is to address an issue I found when parsing columns with only time no date
  #which returned -inf vlaues
  checkyear = np.isinf(mdf_train.iloc[0][column + '_year'])
  if checkyear:
    del mdf_train[column + '_year']
    if column + '_year' in mdf_test.columns:
      del mdf_test[column + '_year']

  checkmonth = np.isinf(mdf_train.iloc[0][column + '_month'])
  if checkmonth:
    del mdf_train[column + '_month']
    if column + '_month' in mdf_test.columns:
      del mdf_test[column + '_month']

  checkday = np.isinf(mdf_train.iloc[0][column + '_day'])
  if checkmonth:
    del mdf_train[column + '_day']
    if column + '_day' in mdf_test.columns:
      del mdf_test[column + '_day']
  
  
  #delete original column from training data
  del mdf_train[column]    
  if column in mdf_test.columns:
    del mdf_test[column]  
  
  #**************************
  #to support the ML infill method, I am updating this function to include an \
  #additional output of a list of the created column names
  
  datecolumns = [column + '_year', column + '_month', column + '_day', \
                column + '_hour', column + '_minute', column + '_second']
  
  #***************
  
  
  
  return mdf_train, mdf_test, datecolumns

In [0]:
#evalcategory(df, column)
#Function that dakes as input a dataframe and associated column id \
#evaluates the contents of cells and classifies the column into one of four categories
#category 1, 'bnry', is for columns with only two categorys of text or integer
#category 2, 'nmbr', is for columns with numerical integer or float values
#category 3, 'text', is for columns with multiple categories appropriate for one-hot
#category 4, 'date', is for columns with Timestamp data
#returns category id as a string

import collections
import datetime as dt

def evalcategory(df, column):
  
  
  #I couldn't find a good pandas tool for evaluating data class, \
  #So will iterate an array through each row of the dataframe column and \
  #evaluation for most common variable using the collections library \
  #this probably isn't extremely efficient for big data scale
  
  #the if/else here is to address a bug I found when iterating through \
  #in a dataframe with single column vs one with multiple columns
  
  array = []
  
  if df.shape[1] > 1:
    for index, row in df.iterrows():
      array = np.append(array, type(row[column]))
      
  else:
    for row in df.iterrows():
      array = np.append(array, type(row[0]))

  c = collections.Counter(array)
  mc = c.most_common(1)
  mc2 = c.most_common(2)
  
  #additional array needed to check for time series
  datearray = []
  
  if df.shape[1] > 1:
    for index, row in df.iterrows():
      datearray = np.append(datearray,type(pd.to_datetime(row[column], errors = 'coerce')))
  
  else:
    for row in df.iterrows():
      datearray = np.append(datearray,type(pd.to_datetime(row[0], errors = 'coerce')))
  
  datec = collections.Counter(datearray)
  datemc = datec.most_common(1)
  datemc2 = datec.most_common(2)
  
  #an extension of this approach could be for those columns that produce a text\
  #category to implement an additional text to determine the number of \
  #common groupings / or the amount of uniquity. For example if every row has\
  #a unique value then one-hot-encoding would not be appropriate. It would \
  #probably be apopropraite to either return an error message if this is found \
  #or alternatively find a furhter way to automate this processing such as \
  #look for contextual clues to groupings that can be inferred.
    
  #This is kind of hack to evaluate class by comparing these with output of mc
  checkint = 1
  checkfloat = 1.1
  checkstring = 'string'
  checkNAN = None

  #there's probably easier way to do this, here will create a check for date
  df_checkdate = pd.DataFrame([{'checkdate' : '7/4/2018'}])
  df_checkdate['checkdate'] = pd.to_datetime(df_checkdate['checkdate'], errors = 'coerce')
  

  #create dummy variable to store determined class (default is text class)
  category = 'text'


  #if most common in column is string and > two values, set category to text
  if isinstance(checkstring, mc[0][0]) and df[column].nunique() > 2:
    category = 'text'
  
  #if most common is date, set category to date
  if isinstance(df_checkdate['checkdate'][0], datemc[0][0]):
    category = 'date'
  
  #if most common in column is integer and > two values, set category to number
  if isinstance(checkint, mc[0][0]) and df[column].nunique() > 2:
    category = 'nmbr'
    
  #if most common in column is float, set category to number
  if isinstance(checkfloat, mc[0][0]):
    category = 'nmbr'
  
  #if most common in column is integer and <= two values, set category to binary
  if isinstance(checkint, mc[0][0]) and df[column].nunique() <= 2:
    category = 'bnry'
  
  #if most common in column is string and <= two values, set category to binary
  if isinstance(checkstring, mc[0][0]) and df[column].nunique() <= 2:
    category = 'bnry'
    
      
  #if most common in column is NaN, re-evaluate using the second most common type
  if 2 * df.isnull().sum()[0] >= df.shape[0]:
    
    #if 2nd most common in column is string and > two values, set category to text
    if isinstance(checkstring, mc2[1][0]) and df[column].nunique() > 2:
      category = 'text'
  
    #if 2nd most common is date, set category to date   
    if isinstance(df_checkdate['checkdate'][0], datemc2[0][0]):
      category = 'date'
  
    #if 2nd most common in column is integer and > two values, set category to number
    if isinstance(checkint, mc2[1][0]) and df[column].nunique() > 2:
      category = 'nmbr'
    
    #if 2nd most common in column is float, set category to number
    if isinstance(checkfloat, mc2[1][0]):
      category = 'nmbr'
  
    #if 2nd most common in column is integer and <= two values, set category to binary
    if isinstance(checkint, mc2[1][0]) and df[column].nunique() <= 2:
      category = 'bnry'
  
    #if 2nd most common in column is string and <= two values, set category to binary
    if isinstance(checkstring, mc2[1][0]) and df[column].nunique() <= 2:
      category = 'bnry'
    
     
  
  return category

# 2) Define new functions

In [0]:
#********************

#NArows(df, column), function that when fed a dataframe, \
#column id, and category label outputs a single column dataframe composed of \
#True and False with the same number of rows as the input and the True's \
#coresponding to those rows of the input that had missing or NaN data. This \
#output can later be used to identify which rows for a column to infill with ML\
# derived plug data


def NArows(df, column, category):
  

  
  if category == 'text':
  
    #returns dataframe of True and False, where True coresponds to the NaN's
    #renames column name to column + '_NArows'
    NArows = pd.isna(df[column])
    NArows = pd.DataFrame(NArows)
    NArows = NArows.rename(columns = {column:column+'_NArows'})
  
  if category == 'bnry':
    
    #returns dataframe of True and False, where True coresponds to the NaN's
    #renames column name to column + '_NArows'
    NArows = pd.isna(df[column])
    NArows = pd.DataFrame(NArows)
    NArows = NArows.rename(columns = {column:column+'_NArows'})
    
  if category == 'nmbr':
  
    #convert all values to either numeric or NaN
    df[column] = pd.to_numeric(df[column], errors='coerce')
    
    #returns dataframe of True and False, where True coresponds to the NaN's
    #renames column name to column + '_NArows'
    NArows = pd.isna(df[column])
    NArows = pd.DataFrame(NArows)
    NArows = NArows.rename(columns = {column:column+'_NArows'})
    
  if category == 'date':
    
    #returns dataframe column of all False
    #renames column name to column + '_NArows'
    NArows = pd.DataFrame(False, index=np.arange(df.shape[0]), columns=[column+'NA'])
    NArows = pd.DataFrame(NArows)
    NArows = NArows.rename(columns = {column:column+'_NArows'})
  

  return NArows


#createMLinfillsets(df_train, df_test, column, trainNArows, testNArows, \
#category, textcolumnslist = []) function that when fed dataframes of train and\
#test sets, column id, df of True/False corresponding to rows from original \
#sets with missing values, a string category of 'text', 'date', 'nmbr', or \
#'bnry', and a list of column id's for the text category. The \
#function returns a seris of dataframes which can be applied to training a \
#machine learning model to predict apppropriate infill values for those points \
#that had missing values from the original sets, indlucing returns of \
#df_train_filltrain, df_train_filllabel, df_train_fillfeatures, \
#and df_test_fillfeatures


def createMLinfillsets(df_train, df_test, column, trainNArows, testNArows, \
                       category, textcolumnslist = []):

  
  #create 3 new dataframes for each train column - the train and labels \
  #for rows not needing infill, and the features for rows needing infill \
  #also create a test features column 

  #reminder:
    #for numerical there won't be a new column
    #for binary there won't be a new column
    #for text the new column has a defined name as column+'_missing'

  #note that for text class the labels will be a little more complicated \
  #since will be multi-column

  if category == 'nmbr' or category == 'bnry':

    #first concatinate the NArows True/False designations to df_train & df_test
    df_train = pd.concat([df_train, trainNArows], axis=1)
    df_test = pd.concat([df_test, testNArows], axis=1)
    
    #create copy of df_train to serve as training set for fill
    df_train_filltrain = df_train.copy()
    #now delete rows coresponding to True
    df_train_filltrain = df_train_filltrain[df_train_filltrain[column+'_NArows'] == False]
    
    #now delete [column] and the NA labels (column+'NA') from this df
    df_train_filltrain = df_train_filltrain.drop([column, column+'_NArows'], axis=1)
    
    #create a copy of df_train[column] for fill train labels
    df_train_filllabel = pd.DataFrame(df_train[column].copy())
    #concatinate with the NArows
    df_train_filllabel = pd.concat([df_train_filllabel, trainNArows], axis=1)
    #drop rows corresponding to True
    df_train_filllabel = df_train_filllabel[df_train_filllabel[column+'_NArows'] == False]
    
    #delete the NArows column
    df_train_filllabel = df_train_filllabel.drop([column+'_NArows'], axis=1)

    #create features df_train for rows needing infill
    #create copy of df_train (note it already has NArows included)
    df_train_fillfeatures = df_train.copy()
    #delete rows coresponding to False
    df_train_fillfeatures = df_train_fillfeatures[(df_train_fillfeatures[column+'_NArows'])]
    #delete column and column+'_NArows'
    df_train_fillfeatures = df_train_fillfeatures.drop([column, column+'_NArows'], axis=1)

    #create features df_test for rows needing infill
    #create copy of df_test (note it already has NArows included)
    df_test_fillfeatures = df_test.copy()
    #delete rows coresponding to False
    df_test_fillfeatures = df_test_fillfeatures[(df_test_fillfeatures[column+'_NArows'])]
    #delete column and column+'_NArows'
    df_test_fillfeatures = df_test_fillfeatures.drop([column, column+'_NArows'], axis=1)
    

    #delete NArows from df_train, df_test
    df_train = df_train.drop([column+'_NArows'], axis=1)
    df_test = df_test.drop([column+'_NArows'], axis=1)


  if category == 'text':

    #first concatinate the NArows True/False designations to df_train & df_test
    df_train = pd.concat([df_train, trainNArows], axis=1)
    df_test = pd.concat([df_test, testNArows], axis=1)

    #create copy of df_train to serve as training set for fill
    df_train_filltrain = df_train.copy()
    #now delete rows coresponding to True
    df_train_filltrain = df_train_filltrain[df_train_filltrain[trainNArows.columns.get_values()[0]] == False]
    
    #now delete columns = textcolumnslist and the NA labels (orig column+'_NArows') from this df
    df_train_filltrain = df_train_filltrain.drop(textcolumnslist, axis=1)
    df_train_filltrain = df_train_filltrain.drop([trainNArows.columns.get_values()[0]], axis=1)

    #create a copy of df_train[textcolumnslist] for fill train labels
    df_train_filllabel = df_train[textcolumnslist].copy()
    #concatinate with the NArows
    df_train_filllabel = pd.concat([df_train_filllabel, trainNArows], axis=1)
    #drop rows corresponding to True
    df_train_filllabel = df_train_filllabel[df_train_filllabel[trainNArows.columns.get_values()[0]] == False]
    
    #delete the NArows column
    df_train_filllabel = df_train_filllabel.drop([trainNArows.columns.get_values()[0]], axis=1)

    #create features df_train for rows needing infill
    #create copy of df_train (note it already has NArows included)
    df_train_fillfeatures = df_train.copy()
    #delete rows coresponding to False
    df_train_fillfeatures = df_train_fillfeatures[(df_train_fillfeatures[trainNArows.columns.get_values()[0]])]
    #delete textcolumnslist and column+'_NArows'
    df_train_fillfeatures = df_train_fillfeatures.drop(textcolumnslist, axis=1)
    df_train_fillfeatures = df_train_fillfeatures.drop([trainNArows.columns.get_values()[0]], axis=1)

    #create features df_test for rows needing infill
    #create copy of df_test (note it already has NArows included)
    df_test_fillfeatures = df_test.copy()
    #delete rows coresponding to False
    df_test_fillfeatures = df_test_fillfeatures[(df_test_fillfeatures[testNArows.columns.get_values()[0]])]
    #delete column and column+'_NArows'
    df_test_fillfeatures = df_test_fillfeatures.drop(textcolumnslist, axis=1)
    df_test_fillfeatures = df_test_fillfeatures.drop([testNArows.columns.get_values()[0]], axis=1)

    #delete NArows from df_train, df_test
    df_train = df_train.drop([trainNArows.columns.get_values()[0]], axis=1)
    df_test = df_test.drop([testNArows.columns.get_values()[0]], axis=1)



  if category == 'date':

    #create empty sets for now
    #an extension of this method would be to implement a comparable method \
    #for the time category, based on the columns output from the preprocessing
    df_train_filltrain = pd.DataFrame({'foo' : []}) 
    df_train_filllabel = pd.DataFrame({'foo' : []})
    df_train_fillfeatures = pd.DataFrame({'foo' : []})
    df_test_fillfeatures = pd.DataFrame({'foo' : []})


  return df_train_filltrain, df_train_filllabel, df_train_fillfeatures, df_test_fillfeatures


#labelbinarizercorrect(npinput, columnslist), function that takes as input the output\
#array from scikit learn's LabelBinarizer() and ensures that the re-encoding is\
#consistent with the original array prior to performing the argmax. This is \
#needed because LabelBinarizer automatically takes two class sets to a binary\
#setting and doesn't account for columns above index of active values based on\
#my understanding. For a large enough dataset this probably won't be an issue \
#but just trying to be thorough. Outputs a one-hot encoded array comparable to \
#the format of our input to argmax.

def labelbinarizercorrect(npinput, columnslist):
  
  
  #if our array post application of LabelBinarizer has few coloumns than our \
  #column list then run through these loops
  if npinput.shape[1] < len(columnslist):
    
    #if only one column in our array means LabelEncoder must have binarized \
    #since we already established that there are more columns
    if npinput.shape[1] == 1:
      
      #this transfers from the binary encoding to two columns of one hot
      npinput = np.hstack((1 - npinput, npinput))
      
      np_corrected = npinput
      
    #if we still have fewer columns than the column list, means we'll need to \
    #pad out with columns containing zeros
    if npinput.shape[1] < len(columnslist):
      missingcols = len(columnslist) - npinput.shape[1]
      append = np.zeros((npinput.shape[0], missingcols))
      np_corrected = np.concatenate((npinput, append), axis=1)
  
  else:
    #otherwise just return the input array because it is in good shape
    np_corrected = npinput

  
  return np_corrected



#predictinfill(category, df_train_filltrain, df_train_filllabel, \
#df_train_fillfeatures, df_test_fillfeatures), function that takes as input \
#a category string, the output of createMLinfillsets(.), and a list of columns \
#produced by a text class preprocessor when applicable and returns \
#predicted infills for the train and test feature sets as df_traininfill, \
#df_testinfill based on derivations using scikit-learn, with the lenth of \
#infill consistent with the number of True values from NArows


#imports for numerical class training
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression

#imports for binary and text class training
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression


def predictinfill(category, df_train_filltrain, df_train_filllabel, \
                  df_train_fillfeatures, df_test_fillfeatures, \
                  textcolumnslist = []):
  
  #a reasonable extension of this funciton would be to allow ML inference with \
  #other ML architectures such a SVM or something SGD based for instance
  
  #convert dataframes to numpy arrays
  np_train_filltrain = df_train_filltrain.values
  np_train_filllabel = df_train_filllabel.values
  np_train_fillfeatures = df_train_fillfeatures.values
  np_test_fillfeatures = df_test_fillfeatures.values
  
  #ony run the following if we have any rows needing infill
  if df_train_fillfeatures.shape[0] > 0:

    if category == 'nmbr':

      #train linear regression model using scikit-learn for numerical prediction
      model = LinearRegression()
      model.fit(np_train_filltrain, np_train_filllabel)

      #predict infill values
      np_traininfill = model.predict(np_train_fillfeatures)
      
      #only run following if we have any test rows needing infill
      if df_test_fillfeatures.shape[0] > 0:
        np_testinfill = model.predict(np_test_fillfeatures)
      else:
        np_testinfill = np.array([0])

      #convert infill values to dataframe
      df_traininfill = pd.DataFrame(np_traininfill, columns = ['infill'])
      df_testinfill = pd.DataFrame(np_testinfill, columns = ['infill'])

#       print('category is nmbr, df_traininfill is')
#       print(df_traininfill)

    if category == 'bnry':

      #train logistic regression model using scikit-learn for binary classifier
      model = LogisticRegression()
      model.fit(np_train_filltrain, np_train_filllabel)

      #predict infill values
      np_traininfill = model.predict(np_train_fillfeatures)
      
      #only run following if we have any test rows needing infill
      if df_test_fillfeatures.shape[0] > 0:
        np_testinfill = model.predict(np_test_fillfeatures)
      else:
        np_testinfill = np.array([0])

      #convert infill values to dataframe
      df_traininfill = pd.DataFrame(np_traininfill, columns = ['infill'])
      df_testinfill = pd.DataFrame(np_testinfill, columns = ['infill'])

#       print('category is bnry, df_traininfill is')
#       print(df_traininfill)

    if category == 'text':

      #first convert the one-hot encoded set via argmax to a 1D array
      np_train_filllabel_argmax = np.argmax(np_train_filllabel, axis=1)

      #train logistic regression model using scikit-learn for binary classifier
      #with multi_class argument activated
      #model = LogisticRegression(multi_class='multinomial')
      model = LogisticRegression()
      model.fit(np_train_filltrain, np_train_filllabel_argmax)

      #predict infill values
      np_traininfill = model.predict(np_train_fillfeatures)
      
      #only run following if we have any test rows needing infill
      if df_test_fillfeatures.shape[0] > 0:
        np_testinfill = model.predict(np_test_fillfeatures)
      else:
        #this needs to have same number of columns as text category
        np_testinfill = np.zeros(shape=(1,len(textcolumnslist)))

      #convert the 1D arrary back to one hot encoding
      labelbinarizertrain = preprocessing.LabelBinarizer()
      labelbinarizertrain.fit(np_traininfill)
      np_traininfill = labelbinarizertrain.transform(np_traininfill)
      
      #only run following if we have any test rows needing infill
      if df_test_fillfeatures.shape[0] > 0:
        labelbinarizertest = preprocessing.LabelBinarizer()
        labelbinarizertest.fit(np_testinfill)
        np_testinfill = labelbinarizertest.transform(np_testinfill)



      #run function to ensure correct dimensions of re-encoded classifier array
      np_traininfill = labelbinarizercorrect(np_traininfill, textcolumnslist)
      
      if df_test_fillfeatures.shape[0] > 0:
        np_testinfill = labelbinarizercorrect(np_testinfill, textcolumnslist)


      #convert infill values to dataframe
      df_traininfill = pd.DataFrame(np_traininfill, columns = [textcolumnslist])
      df_testinfill = pd.DataFrame(np_testinfill, columns = [textcolumnslist]) 


#       print('category is text, df_traininfill is')
#       print(df_traininfill)

    if category == 'date':

      #create empty sets for now
      #an extension of this method would be to implement a comparable infill \
      #method for the time category, based on the columns output from the \
      #preprocessing
      df_traininfill = pd.DataFrame({'infill' : [0]}) 
      df_testinfill = pd.DataFrame({'infill' : [0]}) 

#       print('category is text, df_traininfill is')
#       print(df_traininfill)
  
  
  #else if we didn't have any infill rows let's create some plug values
  else:
    
    if category == 'text':
      np_traininfill = np.zeros(shape=(1,len(textcolumnslist)))
      np_testinfill = np.zeros(shape=(1,len(textcolumnslist)))
      df_traininfill = pd.DataFrame(np_traininfill, columns = [textcolumnslist])
      df_testinfill = pd.DataFrame(np_testinfill, columns = [textcolumnslist]) 
    
    else :
      df_traininfill = pd.DataFrame({'infill' : [0]}) 
      df_testinfill = pd.DataFrame({'infill' : [0]}) 
  
  
  
  return df_traininfill, df_testinfill



#insertinfill(df, column, infill, category, NArows, textcolumnslist = [])
#function that takes as input a dataframe, column id, category string of either\
#'nmbr'/'text'/'bnry'/'date', a df column of True/False identifiying row id of\
#rows that will recieve infill, and and a list of columns produced by a text \
#class preprocessor when applicable. Replaces the column cells in rows \
#coresponding to the NArows True values with the values from infill, returns\
#the associated transformed dataframe.


def insertinfill(df, column, infill, category, NArows, textcolumnslist = []):

  
  
  if category == 'nmbr' or category == 'bnry':
    
    #create new dataframe for infills wherein the infill values are placed in \
    #rows coresponding to NArows True values and rows coresponding to NArows \
    #False values are filled with a 0
    NAarray = []
    i=0
    for index, row in NArows.iterrows():
      if row[column+'_NArows'] == False:
        NAarray = np.append(NAarray, 0)
      if row[column+'_NArows'] == True:
        NAarray = np.append(NAarray, infill.iloc[i]['infill'])
        i += 1
    df_infill_full = pd.DataFrame(NAarray, columns = ['infill'])
      

    #concatinate the dataframes df, NArows, and infill
    df = pd.concat([df, NArows], axis=1)
    df = pd.concat([df, df_infill_full], axis=1)
    
    #for rows where NArows is true, replace value in column column with the \
    #value from infill column
    df.loc[df[column+'_NArows'], column] = df['infill']
    
    #now delete the helper columns
    df = df.drop([column+'_NArows'], axis=1)
    df = df.drop(['infill'], axis=1)
    
    
  if category == 'text':  

    
    #create new dataframe for infills wherein the infill values are placed in \
    #rows coresponding to NArows True values and rows coresponding to NArows \
    #False values are filled with a 0
    
    #text infill contains multiple columns for each predicted calssification
    #which were derived from one-hot encoding the original column in preprocessing
    for textcolumnname in textcolumnslist:
      
      #create newcolumn which will serve as the NArows specific to textcolumnname
      df['textNArows'] = NArows
      
      df['textNArows'] = df['textNArows'].replace(0, False)
      df['textNArows'] = df['textNArows'].replace(1, True)
      
      
      #this will give us an infill array specific to textcolumnname without 0's
      textarray = []
      i=0
      for index, row in df.iterrows():
        if row['textNArows'] == True:
          textarray = np.append(textarray, row[textcolumnname])
          i += 1
      
      #now we'll use a comparable approach as for 'nmbr' and 'bnry'
      NAarray = []
      i=0
      j = infill.columns.get_loc(textcolumnname)
      
      for index, row in df.iterrows():

        if row['textNArows'] == False:
          NAarray = np.append(NAarray, 0)
        if row['textNArows'] == True:
          NAarray = np.append(NAarray, infill.iloc[i][textcolumnname][0])
          i += 1
      df_infill_full = pd.DataFrame(NAarray, columns = ['infill'])
    
      #concatinate the dataframes df, NArows, and infill
      #note we won't need to concatinate the NArows this time since we created\
      #a column specific one called 'textNArows' which is already in place
      df = pd.concat([df, df_infill_full], axis=1)
      
      #for rows where textNArows is true, replace value in column column with \
      #the value from infill column
      df.loc[df['textNArows'], textcolumnname] = df['infill']
      
      #now delete the helper columns
      df = df.drop(['textNArows'], axis=1)
      df = df.drop(['infill'], axis=1)
  
  
  if category == 'date':
    #this spot reserved for future update to incorporate address of datetime\
    #category data
    df = df
  
  return df

# 3) Update automunge(.) function

In [0]:
#automunge(df_train, df_test, labels_column, valpercent=0.20, MLinfill = True)
#Function that when fed a train and test data set automates the process \
#of evaluating each column for determination and applicaiton of appropriate \
#preprocessing. Takes as arguement pandas dataframes of training and test data \
#(mdf_train), (mdf_test), the name of the column from train set containing \
#labels, a string identifying the ID column for train and test, a value for \
#percent of training data to be applied to a validation set, and a True/False \
#selector to determine if MLinfill methods will be applied to any missing \
#points, and a random seed. (If MLinfill = False, missing points are addressed \
#with mean for numerical, most common value for binary, new coluymn for one-hot \
#encoding, and mean for datetime). Note that the ML method for datetime data is \
#future extension. Based on an evaluation of columns selectively applies one of \
#four preprocessing functions to each. Shuffles the data and splits the training \
#set into train and validation sets. Returns following sets as numpy arrays: \
#train, trainID, labels, validation, validationID, validationlabels, test, testID

#Note that this approach assumes that the test data is available at time of training
#A different approach may be required if processing of test data is not simultaneous
#although one potential solution is to apply this function intiially with a dummy\
#dataframe for test set and then when test data becomes available reapply \
#with original train set used for training the model along with the test set.

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

def automunge(df_train, df_test, labels_column, trainID_column = False, \
              testID_column = False, valpercent=0.20, MLinfill = True, \
              randomseed = 42):
  
  #An extension could be to test the input data here for non-dataframe format \
  #(such as csv) to convert it to pandas within the function. 
  
  #my understanding is it is good practice to convert any None values into NaN \
  #so I'll just get that out of the way
  df_train.fillna(value=float('nan'), inplace=True)
  df_test.fillna(value=float('nan'), inplace=True)
  
  #extract the ID columns from train and test set
  if trainID_column != False:
    df_trainID = pd.DataFrame(df_train[trainID_column])
    del df_train[trainID_column]
    
  if testID_column != False:
    df_testID = pd.DataFrame(df_test[testID_column])
    del df_test[testID_column]
  
  #extract labels from train set
  #an extension to this function could be to delete the training set rows\
  #where the labels are missing or improperly formatted prior to performing\
  #this step
  df_labels = pd.DataFrame(df_train[labels_column])
  del df_train[labels_column]
  
  
  #confirm consistency of train an test sets
  
  #check number of columns is consistent
  if df_train.shape[1] != df_test.shape[1]:
    print("error, different number of columns in train and test sets")
    return
  
  #check column headers are consistent (this works independent of order)
  columns_train = set(list(df_train))
  columns_test = set(list(df_test))
  if columns_train != columns_test:
    print("error, different column labels in the train and test set")
    return
   
  #sort columns alphabetically to ensure same order
  #to be honest I'm not positive that this piece is working correctly
  df_train = df_train.sort_index(axis=0)
  df_test = df_test.sort_index(axis=0)
  
  
  #extract column lists again but this time as a list
  columns_train = list(df_train)
  columns_test = list(df_test)

  
  #create an empty dataframe to serve as a store for each column's NArows
  #the column id's for this df will follow convention from NArows of 
  #column+'_NArows' for each column in columns_train
  #these are used in the ML infill methods
  masterNArows_train = pd.DataFrame()
  masterNArows_test = pd.DataFrame()
  
  #create an empty dictionary to serve as a store for each column's category
  #this dictionary will store the key of the original column id with entry of \
  #the associated category string - these are used in the ML infill methods
  mastercategory_dict = {}
  
  #create an empty dictionary to serve as a store specific to the text category
  #our entries to this dictionary will store a master key from each point in \
  #the textcolumns array, with a nested name of original column under \
  #'origcolumn', the full textcolumns array under 'textcolumnsarray', and a \
  #True/False marker we'll call 'infillcomplete' for use in the ML infill methods
  text_dict = {}
  
  #create an empty dictionary to serve as a store specific to the date category
  #our entries to this dictionary will store a master key from each point in \
  #the datecolumns array, with a nested name of original column under \
  #'origcolumn', the full datecolumns array under 'textcolumnsarray', and a \
  #True/False marker we'll call 'infillcomplete' for use in the ML infill methods
  date_dict = {}
  

  
  #For each column, determine appropriate processing function
  #processing function will be based on evaluation of train set
  for column in columns_train:

    category = evalcategory(df_train, column)
    
    #let's make sure the category is consistent between train and test sets
    category_test = evalcategory(df_test, column)
    if category != category_test:
      print('error - different category between train and test sets for column ',\
           column)
    
    
    
    #append this category onto our mastercategory_dict
    mastercategory_dict.update({column+'cat': category})
    
    

    #create NArows (column of True/False where True coresponds to missing data)
    trainNArows = NArows(df_train, column, category)
    testNArows = NArows(df_test, column, category)
    
    #now append that NArows onto a master NA rows df
    masterNArows_train = pd.concat([masterNArows_train, trainNArows], axis=1)
    masterNArows_test = pd.concat([masterNArows_test, testNArows], axis=1)
    

    #(now normalize as would normally)

    

    #for binary class use the majority field for missing plug value
    if category == 'bnry':
      binary_missing_plug = df_train[column].value_counts().index.tolist()[0]
    
    
    #apply appropriate processing function to this column based on the result
    if category == 'bnry':
      df_train = process_binary_class(df_train, column, binary_missing_plug)
      df_test = process_binary_class(df_test, column, binary_missing_plug)
      
    if category == 'nmbr':
      df_train, df_test = process_numerical_class(df_train, df_test, column)
      
    if category == 'text':
      df_train, df_test, textcolumns = process_text_class(df_train, df_test, column)
      

      #store some values in the text_dict{} for use later in ML infill methods
      
      for tc in textcolumns:
        text_dict.update({tc : {'origcolumn' : column, 'textcolumnsarray' : \
                               textcolumns, 'infillcomplete' : False}})

      
    
    if category == 'date':
      df_train, df_test, datecolumns = process_time_class(df_train, df_test, column)
      
      #store some values in the date_dict{} for use later in ML infill methods
      
      for dc in datecolumns:
        date_dict.update({dc : {'origcolumn' : column, 'textcolumnsarray' : \
                               datecolumns, 'infillcomplete' : False}})


  
  #now that we've pre-processed all of the columns, let's run through them again\
  #using ML to derive plug values for the previously missing cells
    
  
  if MLinfill == True:
    
    columns_train_ML = list(df_train)
    columns_test_ML = list(df_test)
    

    for column in columns_train_ML:
      
      #If column id is found in the text_dict then will require different \
      #type of address since this category won't be found in our \
      #mastercategory_dict and we'll need to apply the ML infill to the \
      #collective group of columns from the associated textcolumns array.

      if column in text_dict:

        #check the status of dictionary's infillcomplete marker for this column
        if text_dict[column]['infillcomplete'] == False:

          #pull this column's textcolumns array
          textcolumns = text_dict[column]['textcolumnsarray']

          category = 'text'

          #now let's apply our functions for ML infill

          #createMLinfillsets(df_train, df_test, column, trainNArows, \
          #testNArows, category, textcolumnslist = []), return \
          #df_train_filltrain, df_train_filllabel, df_train_fillfeatures, \
          #df_test_fillfeatures
          df_train_filltrain, df_train_filllabel, df_train_fillfeatures, df_test_fillfeatures = \
          createMLinfillsets(df_train, df_test, column, pd.DataFrame(masterNArows_train[text_dict[column]['origcolumn']+'_NArows']), \
                             pd.DataFrame(masterNArows_test[text_dict[column]['origcolumn']+'_NArows']), category, \
                             textcolumnslist = textcolumns)          


          #predict infill values using defined function predictinfill(.)
          df_traininfill, df_testinfill = \
          predictinfill(category, df_train_filltrain, df_train_filllabel, \
                        df_train_fillfeatures, df_test_fillfeatures, \
                        textcolumnslist = textcolumns)

          #apply the function insertinfill(.) to insert missing value predicitons \
          #to df's associated column
          df_train = insertinfill(df_train, column, df_traininfill, category, \
                                  pd.DataFrame(masterNArows_train[text_dict[column]['origcolumn']+'_NArows']), \
                                  textcolumnslist = textcolumns)

          df_test = insertinfill(df_test, column, df_testinfill, category, \
                                 pd.DataFrame(masterNArows_test[text_dict[column]['origcolumn']+'_NArows']), \
                                 textcolumnslist = textcolumns)

          #now change the infillcomplete marker in the text_dict for each \
          #associated text column
          for textcolumnname in textcolumns:
            text_dict[textcolumnname]['infillcomplete'] = True

      
      #If column id is found in the date_dict then will require different \
      #type of address since this category won't be found in our \
      #mastercategory_dict and we'll need to apply the ML infill to the \
      #collective group of columns from the associated datecolumns array. \
      #The development of this address for date columns is a future extension.
      elif column in date_dict:
        
        #this section to be a future extension.
        pass
      
        #this is for columns that weren't found in the text_dict or date_dict
      else:

        #For each column, determine appropriate processing function
        #processing function will be based on evaluation of train set

        #pull category from dictionary
        category = mastercategory_dict[column+'cat']


        #createMLinfillsets(df_train, df_test, column, trainNArows, testNArows, category, textcolumnslist = [])
        #return df_train_filltrain, df_train_filllabel, df_train_fillfeatures, df_test_fillfeatures

        if category == 'nmbr' or category == 'bnry': #or category == 'date'
          df_train_filltrain, df_train_filllabel, df_train_fillfeatures, df_test_fillfeatures = \
          createMLinfillsets(df_train, df_test, column, pd.DataFrame(masterNArows_train[column+'_NArows']), \
                             masterNArows_test[column+'_NArows'], category)




        #predict infill values using defined function predictinfill(.)

        if category == 'nmbr' or category == 'bnry': #or category == 'date':
          df_traininfill, df_testinfill = \
          predictinfill(category, df_train_filltrain, df_train_filllabel, \
                        df_train_fillfeatures, df_test_fillfeatures, \
                        textcolumnslist = [])



        #apply the function insertinfill(.) to insert missing value predicitons \
        #to df's associated column

        if category == 'nmbr' or category == 'bnry': #or category == 'date':
          df_train = insertinfill(df_train, column, df_traininfill, category, \
                                  pd.DataFrame(masterNArows_train[column+'_NArows']), \
                                  textcolumnslist = [])

          df_test = insertinfill(df_test, column, df_testinfill, category, \
                                 pd.DataFrame(masterNArows_test[column+'_NArows']), \
                                 textcolumnslist = [])



  
  #determine labels category and apply appropriate function
  labelscategory = evalcategory(df_labels, labels_column)
  
  #empty dummy labels "test" df for our preprocessing functions
  labelsdummy = pd.DataFrame()
  
  #apply appropriate processing function to this column based on the result
  if labelscategory == 'bnry':
    labels_binary_missing_plug = df_labels[labels_column].value_counts().index.tolist()[0]
    df_labels = process_binary_class(df_labels, labels_column, labels_binary_missing_plug)
      
  if labelscategory == 'nmbr':
    df_labels, labelsdummy = process_numerical_class(df_labels, labelsdummy, labels_column)
 
  #it occurs to me there might be an argument for preferring a single numerical \
  #classifier for labels to keep this to a single column, if so scikitlearn's \
  #LabelEcncoder could be used here, will assume that onehot encoding is acceptable
  if labelscategory == 'text':
    df_labels, labelsdummy, labelcolumnsdummy = process_text_class(df_labels, labelsdummy, column)
  

  
  #great the data is processed now let's do a few moore global training preps
  
  #convert all of our dataframes to numpy arrays (train, test, labels, and ID)
  #    df_trainID, df_testID
  np_train = df_train.values
  np_test = df_test.values
  np_labels = df_labels.values
  
  if trainID_column != False:
    np_trainID = df_trainID.values
  if testID_column != False:
    np_testID = df_testID.values
  
  
  #set randomness seed number
  answer = randomseed
  #a reasonable extension would be to tie this in with randomness seed for \
  #ML infill methods calls to scikit learn
  
  #shuffle training set and labels
  np_train = shuffle(np_train, random_state = answer)
  np_test = shuffle(np_test, random_state = answer)
  np_labels = shuffle(np_labels, random_state = answer)
  
  if trainID_column != False:
    np_trainID = shuffle(np_trainID, random_state = answer)
  if testID_column != False:
    np_testID = shuffle(np_testID, random_state = answer)
  
  
  #split validation sets from training and labels
  train, validation, labels, validationlabels = \
  train_test_split(np_train, np_labels, test_size=valpercent, shuffle = False)
  
  if trainID_column != False:
    trainID, validationID = \
    train_test_split(np_trainID, test_size=valpercent, shuffle = False)
  else:
    trainID = []
    validationID = []
  if testID_column != False:
    testID = np_testID
  else:
    testID = []
  
  test = np_test
  
  
  #a reasonable extension would be to perform some validation functions on the\
  #sets here (or also prior to transofrm to numpuy arrays) and confirm things \
  #like consistency between format of columns and data between our train and \
  #test sets and if any issues return a coresponding error message to alert user
  
  
  return train, trainID, labels, validation, validationID, validationlabels, test, testID


# 4) Test our functions

In [0]:

#create sample test and train data for demonstration purposes

#train data set from list of dictionaries
#24 rows
train = [{'ID' : 101, 'number': 1, 'Y-N': None, 'shape': 'circle', 'date' : '2/12/18', 'label': 'cat'}, 
         {'ID' : 102, 'number': 2, 'Y-N': 'N', 'shape': 'square', 'date' : 'August 12, 2016', 'label': 'dog'}, 
         {'ID' : 103, 'number': None, 'Y-N': 'Y', 'shape': 'circle', 'date' : None, 'label': 'cat'},
         {'ID' : 104, 'number': 3.1, 'Y-N': None, 'shape': 'square', 'date' : 'July 4, 2016', 'label': 'cat'}, 
         {'ID' : 105, 'number': -1, 'Y-N': None, 'shape': None, 'date' : 'Jul 4, 2018', 'label': 'dog'}, 
         {'ID' : 106, 'number': 'Q', 'Y-N': 'N', 'shape': 'oval', 'date' : '2015', 'label': 'dog'},
         {'ID' : 107, 'number': 1, 'Y-N': None, 'shape': 'circle', 'date' : '2/12/18', 'label': 'cat'}, 
         {'ID' : 108, 'number': 2, 'Y-N': 'N', 'shape': 'square', 'date' : 'August 12, 2016', 'label': 'dog'}, 
         {'ID' : 109, 'number': None, 'Y-N': 'Y', 'shape': 'circle', 'date' : None, 'label': 'cat'},
         {'ID' : 110, 'number': 3.1, 'Y-N': None, 'shape': 'square', 'date' : 'July 4, 2016', 'label': 'cat'}, 
         {'ID' : 111, 'number': -1, 'Y-N': None, 'shape': None, 'date' : 'Jul 4, 2018', 'label': 'dog'}, 
         {'ID' : 112, 'number': 'Q', 'Y-N': None, 'shape': 'oval', 'date' : '2015', 'label': 'dog'},
         {'ID' : 113, 'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : '2/12/18', 'label': 'cat'}, 
         {'ID' : 114, 'number': 2, 'Y-N': None, 'shape': 'square', 'date' : 'August 12, 2016', 'label': 'dog'}, 
         {'ID' : 115, 'number': None, 'Y-N': 'Y', 'shape': 'circle', 'date' : None, 'label': 'cat'},
         {'ID' : 116, 'number': 3.1, 'Y-N': None, 'shape': 'square', 'date' : 'July 4, 2016', 'label': 'cat'}, 
         {'ID' : 117, 'number': -1, 'Y-N': 'N', 'shape': None, 'date' : 'Jul 4, 2018', 'label': 'dog'}, 
         {'ID' : 118, 'number': 'Q', 'Y-N': None, 'shape': 'oval', 'date' : '2015', 'label': 'dog'}]

#convert train data to pandas dataframe
df_train = pd.DataFrame(train)


#test data set from list of dictionaries
#21 rows
test = [{'ID' : 1, 'number': 2.1, 'Y-N': 'N', 'shape': 'square', 'date' : '4/14/18'}, 
        {'ID': 2, 'number': -1, 'Y-N': 'N', 'shape': None, 'date' : 'August 12, 2016'},
        {'ID' : 3,'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : 'July 4, 2018'}, 
        {'ID' : 4, 'number': None, 'Y-N': 'Y', 'shape': 'square', 'date' : None}, 
        {'ID' : 5, 'number': 3, 'Y-N': None, 'shape': 'circle', 'date' : 'Aug 31, 2018'}, 
        {'ID' : 6, 'number': 0, 'Y-N': 'N', 'shape': 'octogon', 'date' : '2017'}, 
        {'ID' : 7, 'number': 'Q', 'Y-N': 'Y', 'shape': 'square', 'date' : 'Jan 1, 2019'},
        {'ID' : 8, 'number': 2.1, 'Y-N': 'N', 'shape': 'square', 'date' : '4/14/18'}, 
        {'ID' : 9, 'number': -1, 'Y-N': 'N', 'shape': None, 'date' : 'August 12, 2016'},
        {'ID' : 10, 'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : 'July 4, 2018'}, 
        {'ID' : 11, 'number': None, 'Y-N': 'Y', 'shape': 'square', 'date' : None}, 
        {'ID' : 12, 'number': 3, 'Y-N': None, 'shape': 'circle', 'date' : 'Aug 31, 2018'}, 
        {'ID' : 13, 'number': 0, 'Y-N': 'N', 'shape': 'octogon', 'date' : '2017'}, 
        {'ID' : 14, 'number': 'Q', 'Y-N': 'Y', 'shape': 'square', 'date' : 'Jan 1, 2019'},
        {'ID' : 15, 'number': 2.1, 'Y-N': 'N', 'shape': 'square', 'date' : '4/14/18'}, 
        {'ID' : 16, 'number': -1, 'Y-N': 'N', 'shape': None, 'date' : 'August 12, 2016'},
        {'ID' : 17, 'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : 'July 4, 2018'}, 
        {'ID' : 18, 'number': None, 'Y-N': 'Y', 'shape': 'square', 'date' : None}, 
        {'ID' : 19, 'number': 3, 'Y-N': None, 'shape': 'circle', 'date' : 'Aug 31, 2018'}, 
        {'ID' : 20, 'number': 0, 'Y-N': 'N', 'shape': 'octogon', 'date' : '2017'}, 
        {'ID' : 21, 'number': 'Q', 'Y-N': 'Y', 'shape': 'square', 'date' : 'Jan 1, 2019'}]

#convert test data to pandas dataframe
df_test = pd.DataFrame(test)



In [0]:
#apply automunge

train, trainID, labels, validation, validationID, validationlabels, test, testID = \
automunge(df_train, df_test, labels_column = 'label', trainID_column = 'ID', \
         testID_column = 'ID', MLinfill = True)

In [0]:
train

array([[ 0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
        -0.21691867,  1.12710708, -1.        ,  1.14830512,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.5718765 , -0.48304589,  1.        ,  1.14830512,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         0.88586843,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         2.9819328 , -1.28812238, -1.33333333, -1.19083494,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         1.43955119, -0.48304589,  0.66666667, -0.55288765,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.5718765 , -

In [0]:

#create sample test and train data for demonstration purposes

#train data set from list of dictionaries
#24 rows
train = [{'ID' : 101, 'number': 1, 'Y-N': None, 'shape': 'circle', 'date' : '2/12/18', 'label': 'cat'}, 
         {'ID' : 102, 'number': 2, 'Y-N': 'N', 'shape': 'square', 'date' : 'August 12, 2016', 'label': 'dog'}, 
         {'ID' : 103, 'number': None, 'Y-N': 'Y', 'shape': 'circle', 'date' : None, 'label': 'cat'},
         {'ID' : 104, 'number': 3.1, 'Y-N': None, 'shape': 'square', 'date' : 'July 4, 2016', 'label': 'cat'}, 
         {'ID' : 105, 'number': -1, 'Y-N': None, 'shape': None, 'date' : 'Jul 4, 2018', 'label': 'dog'}, 
         {'ID' : 106, 'number': 'Q', 'Y-N': 'N', 'shape': 'oval', 'date' : '2015', 'label': 'dog'},
         {'ID' : 107, 'number': 1, 'Y-N': None, 'shape': 'circle', 'date' : '2/12/18', 'label': 'cat'}, 
         {'ID' : 108, 'number': 2, 'Y-N': 'N', 'shape': 'square', 'date' : 'August 12, 2016', 'label': 'dog'}, 
         {'ID' : 109, 'number': None, 'Y-N': 'Y', 'shape': 'circle', 'date' : None, 'label': 'cat'},
         {'ID' : 110, 'number': 3.1, 'Y-N': None, 'shape': 'square', 'date' : 'July 4, 2016', 'label': 'cat'}, 
         {'ID' : 111, 'number': -1, 'Y-N': None, 'shape': None, 'date' : 'Jul 4, 2018', 'label': 'dog'}, 
         {'ID' : 112, 'number': 'Q', 'Y-N': None, 'shape': 'oval', 'date' : '2015', 'label': 'dog'},
         {'ID' : 113, 'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : '2/12/18', 'label': 'cat'}, 
         {'ID' : 114, 'number': 2, 'Y-N': None, 'shape': 'square', 'date' : 'August 12, 2016', 'label': 'dog'}, 
         {'ID' : 115, 'number': None, 'Y-N': 'Y', 'shape': 'circle', 'date' : None, 'label': 'cat'},
         {'ID' : 116, 'number': 3.1, 'Y-N': None, 'shape': 'square', 'date' : 'July 4, 2016', 'label': 'cat'}, 
         {'ID' : 117, 'number': -1, 'Y-N': 'N', 'shape': None, 'date' : 'Jul 4, 2018', 'label': 'dog'}, 
         {'ID' : 118, 'number': 'Q', 'Y-N': None, 'shape': 'oval', 'date' : '2015', 'label': 'dog'}]

#convert train data to pandas dataframe
df_train = pd.DataFrame(train)


#test data set from list of dictionaries
#21 rows
test = [{'ID' : 1, 'number': 2.1, 'Y-N': 'N', 'shape': 'square', 'date' : '4/14/18'}, 
        {'ID': 2, 'number': -1, 'Y-N': 'N', 'shape': None, 'date' : 'August 12, 2016'},
        {'ID' : 3,'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : 'July 4, 2018'}, 
        {'ID' : 4, 'number': None, 'Y-N': 'Y', 'shape': 'square', 'date' : None}, 
        {'ID' : 5, 'number': 3, 'Y-N': None, 'shape': 'circle', 'date' : 'Aug 31, 2018'}, 
        {'ID' : 6, 'number': 0, 'Y-N': 'N', 'shape': 'octogon', 'date' : '2017'}, 
        {'ID' : 7, 'number': 'Q', 'Y-N': 'Y', 'shape': 'square', 'date' : 'Jan 1, 2019'},
        {'ID' : 8, 'number': 2.1, 'Y-N': 'N', 'shape': 'square', 'date' : '4/14/18'}, 
        {'ID' : 9, 'number': -1, 'Y-N': 'N', 'shape': None, 'date' : 'August 12, 2016'},
        {'ID' : 10, 'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : 'July 4, 2018'}, 
        {'ID' : 11, 'number': None, 'Y-N': 'Y', 'shape': 'square', 'date' : None}, 
        {'ID' : 12, 'number': 3, 'Y-N': None, 'shape': 'circle', 'date' : 'Aug 31, 2018'}, 
        {'ID' : 13, 'number': 0, 'Y-N': 'N', 'shape': 'octogon', 'date' : '2017'}, 
        {'ID' : 14, 'number': 'Q', 'Y-N': 'Y', 'shape': 'square', 'date' : 'Jan 1, 2019'},
        {'ID' : 15, 'number': 2.1, 'Y-N': 'N', 'shape': 'square', 'date' : '4/14/18'}, 
        {'ID' : 16, 'number': -1, 'Y-N': 'N', 'shape': None, 'date' : 'August 12, 2016'},
        {'ID' : 17, 'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : 'July 4, 2018'}, 
        {'ID' : 18, 'number': None, 'Y-N': 'Y', 'shape': 'square', 'date' : None}, 
        {'ID' : 19, 'number': 3, 'Y-N': None, 'shape': 'circle', 'date' : 'Aug 31, 2018'}, 
        {'ID' : 20, 'number': 0, 'Y-N': 'N', 'shape': 'octogon', 'date' : '2017'}, 
        {'ID' : 21, 'number': 'Q', 'Y-N': 'Y', 'shape': 'square', 'date' : 'Jan 1, 2019'}]

#convert test data to pandas dataframe
df_test = pd.DataFrame(test)



In [0]:
#apply automunge without ML infill, compare results


train, trainID, labels, validation, validationID, validationlabels, test, testID = \
automunge(df_train, df_test, labels_column = 'label',  trainID_column = 'ID', \
         testID_column = 'ID', MLinfill = False)

In [0]:
train

array([[ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        -0.21691867,  1.12710708, -1.        ,  1.14830512,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.5718765 , -0.48304589,  1.        ,  1.14830512,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        , -1.28812238, -1.33333333, -1.19083494,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         1.43955119, -0.48304589,  0.66666667, -0.55288765,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.5718765 , -

In [0]:
#Now let's try a larger dataset, the Titanic dataset from Kaggle
#available here: https://www.kaggle.com/c/titanic/data
#(which I will upload form my local hard drive)
#for more on data imports in Colaboratory see my medium post 
#https://medium.com/@_NicT_/colaboratorys-free-gpu-72ebc9272933
#Following is as presented in the Colaboratory tutorial notebook
#Once run this will allow you to manually select the path on local drive for file you wish to upload

from google.colab import files
uploaded = files.upload()
for train in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=train, length=len(uploaded[train])))


Saving train.csv to train (1).csv
User uploaded file "train.csv" with length 60302 bytes


In [0]:
#Here is some additional detail for converting 
#the resulting upload into a dataframe

from io import BytesIO
titanic_train_dforig = pd.read_csv(BytesIO(uploaded[train]), encoding='latin-1')
titanic_train_dforig.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [0]:
from google.colab import files
uploaded = files.upload()
for train in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=train, length=len(uploaded[train])))


Saving test.csv to test (1).csv
User uploaded file "test.csv" with length 28210 bytes


In [0]:
#Here is some additional detail for converting 
#the resulting upload into a dataframe

from io import BytesIO
titanic_test_dforig = pd.read_csv(BytesIO(uploaded[train]), encoding='latin-1')
titanic_test_dforig.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [0]:
titanic_train_df = titanic_train_dforig.copy()
titanic_test_df = titanic_test_dforig.copy()

In [0]:
#Now there are certain aspects of feature engineering that our automunge won't address
#for example one could extract from the Mrs/Ms/Miss designation in the Name \
#column if a female is married. From Cabin field perhaps we could infer what \
#deck passenger was on or whether they even had a cabin. This type of evaluation \
#would need to be done prior to applicaiton of automunge. Because each column is \
#unique there won't be any learning for Cabin, Name, or Ticket I expect so we'll \
#go ahead and delete those rows for our demonstration. It is certainly \
#feasible that there is some feature buried in these columns that can be \
#extracted prior to applicaiton of automunge. PassengerId will serve as ID column.

titanic_train_df = titanic_train_df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
titanic_test_df = titanic_test_df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [0]:
#now let's run our automunge function and see how we did, first we'll try \
#without the MLinfill:

train, trainID, labels, validation, validationID, validationlabels, test, testID = \
automunge(titanic_train_df, titanic_test_df, labels_column = 'Survived', \
          trainID_column = 'PassengerId', testID_column = 'PassengerId', \
          MLinfill = False)


In [0]:
train

array([[ 1.        ,  0.        ,  0.        , ...,  0.43255043,
         0.76719899, -0.34126057],
       [ 0.        ,  0.        ,  1.        , ..., -0.47427882,
        -0.47340772, -0.43676213],
       [ 0.        ,  0.        ,  1.        , ..., -0.47427882,
        -0.47340772, -0.48857985],
       ...,
       [ 0.        ,  0.        ,  1.        , ..., -0.47427882,
        -0.47340772, -0.48916745],
       [ 0.        ,  0.        ,  1.        , ...,  3.15303818,
         0.76719899,  0.15058917],
       [ 1.        ,  0.        ,  0.        , ..., -0.47427882,
         0.76719899, -0.47667284]])

In [0]:
#re-initialize the data

titanic_train_df = titanic_train_dforig.copy()
titanic_test_df = titanic_test_dforig.copy()

titanic_train_df = titanic_train_df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
titanic_test_df = titanic_test_df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [0]:
#now let's run our automunge function and see how we did with the MLinfill:

train, trainID, labels, validation, validationID, validationlabels, test, testID = \
automunge(titanic_train_df, titanic_test_df, labels_column = 'Survived', \
          trainID_column = 'PassengerId', testID_column = 'PassengerId', \
          MLinfill = True)


In [0]:
train

array([[ 1.        ,  0.        ,  0.        , ...,  0.43255043,
         0.76719899, -0.34126057],
       [ 0.        ,  0.        ,  1.        , ..., -0.47427882,
        -0.47340772, -0.43676213],
       [ 0.        ,  0.        ,  1.        , ..., -0.47427882,
        -0.47340772, -0.48857985],
       ...,
       [ 0.        ,  0.        ,  1.        , ..., -0.47427882,
        -0.47340772, -0.48916745],
       [ 0.        ,  0.        ,  1.        , ...,  3.15303818,
         0.76719899,  0.15058917],
       [ 1.        ,  0.        ,  0.        , ..., -0.47427882,
         0.76719899, -0.47667284]])